# Content based recommender system

In this notebook you will learn how to build a content based recommender-system...

requirements:

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
#from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

preprosessed advocate df from [here](https://github.com/ruthgn/Beer-Recommender):

features are:

- Alcohol content (% by volume)
- Minimum and maximum IBU (International Bitterness Units)
- Mouthfeel:
    - Astringency
    - Body
    - Alcohol
- Taste:
    - Bitter
    - Sweet
    - Sour
    - Salty
- Flavor And Aroma:
    - Fruits
    - Hoppy
    - Spices
    - Malty

In [4]:

df = pd.read_csv("data/updated_beer_profile_and_ratings.csv")
df.head()

,Name,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
0,Amber,Altbier,Alaskan Brewing Co.,Alaskan Brewing Co. Alaskan Amber,"Notes:Richly malty and long on the palate, wit...",5.3,25,50,13,32,...,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082,497
1,Double Bag,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Double Bag,"Notes:This malty, full-bodied double alt is al...",7.2,25,50,12,57,...,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304,481
2,Long Trail Ale,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Long Trail Ale,Notes:Long Trail Ale is a full-bodied amber al...,5.0,25,50,14,37,...,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239,377
3,Doppelsticke,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Notes:,8.5,25,50,13,55,...,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435,368
4,Sleigh'r Dark Doüble Alt Ale,Altbier,Ninkasi Brewing Company,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Notes:Called 'Dark Double Alt' on the label.Se...,7.2,25,50,25,51,...,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708,96


In [5]:
df.shape

(3197, 25)

ratebeer df from [here](https://www.kaggle.com/dfsets/ehallmar/beers-breweries-and-beer-reviews?select=reviews.csv):

The ratebeer dataset could be interesting because it contains written descriptions of beer. 

In [6]:
#read data
df_rb = pd.read_csv("data_ratebeer/reviews.csv")
df_rb.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.00,4.00,4.00,4.25,4.00,4.03
1,125646,_dirty_,2017-12-21,,4.50,4.50,4.50,4.50,4.50,4.50
2,125646,CJDUBYA,2017-12-21,,4.75,4.75,4.75,4.75,4.75,4.75
3,125646,GratefulBeerGuy,2017-12-20,0% 16 oz can. Funny story: As I finally wal...,4.75,4.75,4.50,4.50,4.50,4.58
4,125646,LukeGude,2017-12-20,Classic TH NEIPA. Overflowing head and bouq...,4.25,4.50,4.25,4.25,4.25,4.31


Let's see what the descriptions look like: 

In [7]:
df_rb["text"][:5]

0       750 ml bottle, 2016 vintage, bottle #304 of...
1                                                     
2                                                     
3       0% 16 oz can. Funny story: As I finally wal...
4       Classic TH NEIPA. Overflowing head and bouq...
Name: text, dtype: object

In [8]:
df_rb["text"].iloc[0]

"\xa0\xa0 750 ml bottle, 2016 vintage, bottle #304 of 360. Served in a DFH snifter, the beer pours dark brown/black with about an inch tan head that stuck around a little while. There's also a good amount of lacing. I like the aroma, the brew smells like dark fruit (raisin, plum), chocolate, roasted malt, and a bit of licorice. For the most part I think the taste is just like the aroma, but with some coffee flavor noticeable too. Mouthfeel/body is medium/full, it's creamy and coating with a moderate amount of carbonation. I think it's a good overall brew, worth picking up if you want a nice straight forward imperial stout. Here's hoping Four String brings this back in the future! $9.99 a bottle. \xa0"

In [9]:
df_rb.shape

(9073128, 10)

The ratebeer df contains, apart from beer characteristics, user reviews in text form. Those might be useful or not. Regarding the fact that this df set has 9 Mio entries, I will stay with the beer advocate df for now.

## 1. data cleaning and EDA

A little df overview:

In [10]:
df.shape 

(3197, 25)

In [11]:
df.describe()

,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
count,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000
mean,6.526688,21.180482,38.986863,16.515796,46.129496,17.055990,36.364404,58.270879,33.145449,1.017204,38.529559,40.924617,18.345637,75.330935,3.638789,3.754393,3.660428,3.702496,3.749690,233.284955
std,2.546997,13.242242,21.355281,10.410661,25.947842,17.331334,25.791152,34.281310,35.780172,2.132651,32.296646,30.403641,23.756582,39.909338,0.503209,0.403416,0.449937,0.510361,0.387217,361.811847
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.509615,1.571429,1.285714,1.214286,1.471154,1.000000
25%,5.000000,15.000000,25.000000,9.000000,29.000000,6.000000,17.000000,33.000000,11.000000,0.000000,12.000000,18.000000,4.000000,45.000000,3.422559,3.604651,3.470021,3.500000,3.622449,23.000000
50%,6.000000,20.000000,35.000000,14.000000,40.000000,11.000000,31.000000,54.000000,22.000000,0.000000,29.000000,33.000000,10.000000,73.000000,3.720183,3.833333,3.741667,3.791667,3.825397,93.000000
75%,7.600000,25.000000,45.000000,21.000000,58.000000,22.000000,52.000000,77.000000,42.000000,1.000000,60.000000,56.000000,23.000000,103.000000,3.978000,4.000000,3.965587,4.033333,3.995192,284.000000
max,57.500000,65.000000,100.000000,81.000000,175.000000,139.000000,150.000000,263.000000,284.000000,48.000000,175.000000,172.000000,184.000000,239.000000,5.000000,4.666667,5.000000,5.000000,4.634615,3290.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3197 entries, 0 to 3196
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               3197 non-null   object 
 1   Style              3197 non-null   object 
 2   Brewery            3197 non-null   object 
 3   Beer Name (Full)   3197 non-null   object 
 4   Description        3197 non-null   object 
 5   ABV                3197 non-null   float64
 6   Min IBU            3197 non-null   int64  
 7   Max IBU            3197 non-null   int64  
 8   Astringency        3197 non-null   int64  
 9   Body               3197 non-null   int64  
 10  Alcohol            3197 non-null   int64  
 11  Bitter             3197 non-null   int64  
 12  Sweet              3197 non-null   int64  
 13  Sour               3197 non-null   int64  
 14  Salty              3197 non-null   int64  
 15  Fruits             3197 non-null   int64  
 16  Hoppy              3197 

In [13]:
#remove duplicates
df.drop_duplicates(inplace = True)

In [14]:
df.shape

(3197, 25)

Since the shape is still the same, the dataframe did not contain duplicates. 

We want to do our recommendations based on the Name variable, let's see how many unique beer names there are.

In [15]:
df.Name.nunique() 

3066

There is a difference of 131 between number of rows and unique Name values which means different breweries named their beer the same. This can cause problems while trying to give recommendations. To solve this issue I combine the Name with the name of the Brewery. 

In [16]:
df["unique_name"] = df.Name + " " + "(" + df.Brewery + ")"
df.unique_name

0                             Amber (Alaskan Brewing Co.)
1                     Double Bag (Long Trail Brewing Co.)
2                 Long Trail Ale (Long Trail Brewing Co.)
3       Doppelsticke (Uerige Obergärige Hausbrauerei G...
4       Sleigh'r Dark Doüble Alt Ale (Ninkasi Brewing ...
                              ...                        
3192                 Winter Shredder (Cisco Brewers Inc.)
3193      The First Snow Ale (RJ Rockers Brewing Company)
3194    Red Nose Winter Ale (Natty Greene's Pub & Brew...
3195    Fish Tale Winterfish (Fish Brewing Company / F...
3196    Frosted Frog Christmas Ale - Barrel-Aged (Hopp...
Name: unique_name, Length: 3197, dtype: object

In [17]:
df.unique_name.nunique()

3197

Now the unuique names equal the number of rows.

## 2. Simple model based on discription

In [18]:
df.head()

,Name,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews,unique_name
0,Amber,Altbier,Alaskan Brewing Co.,Alaskan Brewing Co. Alaskan Amber,"Notes:Richly malty and long on the palate, wit...",5.3,25,50,13,32,...,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082,497,Amber (Alaskan Brewing Co.)
1,Double Bag,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Double Bag,"Notes:This malty, full-bodied double alt is al...",7.2,25,50,12,57,...,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304,481,Double Bag (Long Trail Brewing Co.)
2,Long Trail Ale,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Long Trail Ale,Notes:Long Trail Ale is a full-bodied amber al...,5.0,25,50,14,37,...,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239,377,Long Trail Ale (Long Trail Brewing Co.)
3,Doppelsticke,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Notes:,8.5,25,50,13,55,...,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435,368,Doppelsticke (Uerige Obergärige Hausbrauerei G...
4,Sleigh'r Dark Doüble Alt Ale,Altbier,Ninkasi Brewing Company,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Notes:Called 'Dark Double Alt' on the label.Se...,7.2,25,50,25,51,...,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708,96,Sleigh'r Dark Doüble Alt Ale (Ninkasi Brewing ...


In [19]:
df.Name[:5]

0                           Amber
1                      Double Bag
2                  Long Trail Ale
3                    Doppelsticke
4    Sleigh'r Dark Doüble Alt Ale
Name: Name, dtype: object

In [21]:
#vectorize description column
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['Description'])

In [42]:
#calculate cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.00000000e+00, 3.22311393e-02, 4.62455991e-02, ...,
        2.61037018e-02, 2.61037018e-02, 5.44062446e-04],
       [3.22311393e-02, 1.00000000e+00, 1.14964677e-01, ...,
        1.11189813e-02, 1.11189813e-02, 8.46053443e-03],
       [4.62455991e-02, 1.14964677e-01, 1.00000000e+00, ...,
        1.49214249e-02, 1.49214249e-02, 3.10997536e-04],
       ...,
       [2.61037018e-02, 1.11189813e-02, 1.49214249e-02, ...,
        1.00000000e+00, 1.00000000e+00, 2.08423483e-02],
       [2.61037018e-02, 1.11189813e-02, 1.49214249e-02, ...,
        1.00000000e+00, 1.00000000e+00, 2.08423483e-02],
       [5.44062446e-04, 8.46053443e-03, 3.10997536e-04, ...,
        2.08423483e-02, 2.08423483e-02, 1.00000000e+00]])

In [44]:
# Build a 1-dimensional array with beer names
names = df['unique_name']
indices = pd.Series(df.index, index=df['unique_name'])

# Function that get book recommendations based on the cosine similarity 
def beer_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    beer_indices = [i[0] for i in sim_scores]
    return names.iloc[beer_indices]

In [45]:
#get recommendations
beer_recommendations('Amber (Alaskan Brewing Co.)')

28              Crooked Coast Altbier (Driftwood Brewery)
2217                        Gaelic Ale (Highland Brewing)
4       Sleigh'r Dark Doüble Alt Ale (Ninkasi Brewing ...
2524                   Obsidian Stout (Deschutes Brewery)
38                   The Detroit Dwarf (Detroit Beer Co.)
912     Old East India Pale Ale (Sea Dog Brewing Company)
525     Rio Grande Outlaw LagerSierra Blanca Brewing C...
1284             Penn Gold (Pennsylvania Brewing Company)
1563    Jenny Lake Lager (Snake River Brewing Company ...
1860    Smuttynose Shoals Pale Ale (Smuttynose Brewing...
424       Moose Drool Brown Ale (Big Sky Brewing Company)
2237                         Mourning Wood (Local Option)
17                   Headwall Alt (Tuckerman Brewing Co.)
1810                    Doggie Style (Flying Dog Brewery)
1578               Red Stallion (Crescent City Brewhouse)
3145    Framboise For A Cure (Russian River Brewing Co...
1922                          Pils (Heater Allen Brewing)
1063          

In [25]:
# check available features
df.columns

Index(['Name', 'Style', 'Brewery', 'Beer Name (Full)', 'Description', 'ABV',
       'Min IBU', 'Max IBU', 'Astringency', 'Body', 'Alcohol', 'Bitter',
       'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty',
       'review_aroma', 'review_appearance', 'review_palate', 'review_taste',
       'review_overall', 'number_of_reviews', 'unique_name'],
      dtype='object')

### reommender system based on other features than description

In [29]:
#feature matrix
matrix = df.iloc[:,5:-1]

In [40]:
#calculate cosine similarity
cosine_sim = linear_kernel(matrix, matrix)
cosine_sim

# Build a 1-dimensional array with beer names
names = df['unique_name']
indices = pd.Series(df.index, index=df['unique_name'])

# Function that get beer recommendations based on the cosine similarity 
def beer_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    beer_indices = [i[0] for i in sim_scores]
    return df.iloc[beer_indices]

beer_recommendations('Amber (Alaskan Brewing Co.)')

,Name,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews,unique_name
803,Celebration Fresh Hop IPA,IPA,Sierra Nevada Brewing Co.,Sierra Nevada Brewing Co. Sierra Nevada Celebr...,Notes:Fresh Hop Ale; no spices.We first brewed...,6.8,50,70,19,38,...,92,5,47,4.082333,4.225167,4.076000,4.190000,4.168667,3000,Celebration Fresh Hop IPA (Sierra Nevada Brewi...
955,Ruination Double IPA 2.0,IPA,Stone Brewing Co.,Stone Brewing Co. Stone Ruination IPA,Notes:Also known as Ruination in Europe.A Liqu...,8.5,65,100,21,37,...,135,0,26,4.341531,4.179179,4.183432,4.351331,4.161612,2704,Ruination Double IPA 2.0 (Stone Brewing Co.)
925,Pliny The Elder,IPA,Russian River Brewing Company,Russian River Brewing Company Pliny The Elder,"Notes:Pliny the Elder is brewed with Amarillo,...",8.0,65,100,26,29,...,85,1,32,4.612188,4.388603,4.451326,4.630985,4.590028,2527,Pliny The Elder (Russian River Brewing Company)
2658,Breakfast Stout,Stout,Founders Brewing Company,Founders Brewing Company Founders Breakfast Stout,Notes:The coffee lover’s consummate beer. Brew...,8.3,20,40,11,96,...,22,12,165,4.412070,4.364508,4.387890,4.501199,4.354516,2502,Breakfast Stout (Founders Brewing Company)
2891,La Fin Du Monde,Tripel,Unibroue,Unibroue La Fin Du Monde,Notes:La Fin du Monde was developed through 18...,9.0,20,40,14,41,...,31,53,36,4.231977,4.193919,4.264398,4.396093,4.298027,2483,La Fin Du Monde (Unibroue)
2496,Storm King Stout,Stout,Victory Brewing Company,Victory Brewing Company Storm King Stout,Notes:Emerging from the deepest shades of dark...,9.1,50,80,6,70,...,84,9,95,4.070555,4.304445,4.138458,4.211664,3.984787,2452,Storm King Stout (Victory Brewing Company)
801,60 Minute IPA,IPA,Dogfish Head Brewery,Dogfish Head Brewery 60 Minute IPA,Notes:60 Minute IPA is continuously hopped -- ...,6.0,50,70,17,26,...,75,3,24,3.996768,3.966061,4.006667,4.118990,4.172727,2475,60 Minute IPA (Dogfish Head Brewery)
2693,Black Chocolate Stout,Stout,Brooklyn Brewery,Brooklyn Brewery Brooklyn Black Chocolate Stout,Notes:This is the famous Brooklyn Black Chocol...,10.0,50,90,12,107,...,30,10,111,4.110544,4.293829,4.165304,4.220883,4.032284,2447,Black Chocolate Stout (Brooklyn Brewery)
2816,Duvel,Strong Ale,Brouwerij Duvel Moortgat NV,Brouwerij Duvel Moortgat NV Duvel,Notes:A Duvel is still seen as the reference a...,8.5,20,40,19,31,...,49,31,27,4.151224,4.512857,4.257347,4.329592,4.342245,2450,Duvel (Brouwerij Duvel Moortgat NV)
1546,Samuel Adams Boston LagerBoston Beer Company (...,Lager,Boston Beer Company (Samuel Adams),Boston Beer Company (Samuel Adams) Samuel Adam...,Notes:Four Ingredients. Painstakingly Chosen. ...,5.0,15,30,17,25,...,54,7,83,3.663358,3.886476,3.774194,3.872415,4.044251,2418,Samuel Adams Boston LagerBoston Beer Company (...


### scaled data?

In [31]:
#scale matrix
scaler = MinMaxScaler()
scaled_matrix = scaler.fit_transform(matrix)
scaled_matrix

array([[0.09217391, 0.38461538, 0.5       , ..., 0.64177511, 0.75105336,
        0.15080572],
       [0.12521739, 0.38461538, 0.5       , ..., 0.74243907, 0.810236  ,
        0.14594102],
       [0.08695652, 0.38461538, 0.5       , ..., 0.63845658, 0.74572912,
        0.11432046],
       ...,
       [0.11826087, 0.53846154, 0.5       , ..., 0.64949201, 0.70820661,
        0.00760109],
       [0.13043478, 0.53846154, 0.5       , ..., 0.71004122, 0.76124093,
        0.02614777],
       [0.14956522, 0.53846154, 0.5       , ..., 0.61844873, 0.72931404,
        0.00516874]])

In [41]:
#calculate cosine similarity
cosine_sim = linear_kernel(scaled_matrix, scaled_matrix)
cosine_sim

# Build a 1-dimensional array with beer names
names = df['unique_name']
indices = pd.Series(df.index, index=df['unique_name'])

# Function that get beer recommendations based on the cosine similarity 
def beer_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    beer_indices = [i[0] for i in sim_scores]
    return df.iloc[beer_indices]

beer_recommendations('Amber (Alaskan Brewing Co.)')

,Name,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews,unique_name
2489,KBS (Kentucky Breakfast Stout),Stout,Founders Brewing Company,Founders Brewing Company Founders KBS (Kentuck...,Notes:What we’ve got here is an imperial stout...,12.30,50,80,9,114,...,31,20,183,4.570393,4.377070,4.479814,4.626294,4.397516,1932,KBS (Kentucky Breakfast Stout) (Founders Brewi...
925,Pliny The Elder,IPA,Russian River Brewing Company,Russian River Brewing Company Pliny The Elder,"Notes:Pliny the Elder is brewed with Amarillo,...",8.00,65,100,26,29,...,85,1,32,4.612188,4.388603,4.451326,4.630985,4.590028,2527,Pliny The Elder (Russian River Brewing Company)
944,Pliny The Younger,IPA,Russian River Brewing Company,Russian River Brewing Company Pliny The Younger,"Notes:Pliny the Younger, the man, was Pliny th...",10.25,65,100,23,46,...,106,4,19,4.723770,4.482787,4.612295,4.724590,4.600000,610,Pliny The Younger (Russian River Brewing Company)
2241,Nugget Nectar,Red Ale,Tröegs Brewing Company,Tröegs Brewing Company Tröegs Nugget Nectar,Notes:Squeeze those hops for all they're worth...,7.50,55,85,28,41,...,139,13,90,4.371100,4.212276,4.198210,4.413555,4.395652,1955,Nugget Nectar (Tröegs Brewing Company)
41,Old Ruffian,Barleywine,Great Divide Brewing Company,Great Divide Brewing Company Old Ruffian Barle...,"Notes:Old Ruffian is a hefty, hop-forward barl...",10.20,60,100,6,55,...,120,14,119,4.238552,4.318956,4.231629,4.285942,4.071885,939,Old Ruffian (Great Divide Brewing Company)
954,Double Sunshine,IPA,Lawson's Finest Liquids,Lawson's Finest Liquids Double Sunshine IPA,Notes:This American double India pale ale is p...,8.00,65,100,17,32,...,102,4,30,4.594118,4.317647,4.311765,4.552941,4.588235,85,Double Sunshine (Lawson's Finest Liquids)
43,Behemoth,Barleywine,Three Floyds Brewing Co. & Brewpub,Three Floyds Brewing Co. & Brewpub Behemoth Bl...,"Notes:A huge, sweet barleywine with complex ca...",10.50,60,100,18,71,...,105,24,105,4.207373,4.123272,4.187788,4.264977,4.059908,434,Behemoth (Three Floyds Brewing Co. & Brewpub)
2714,Imperial Stout,Stout,Rogue Ales,Rogue Ales Imperial Stout,Notes:Also known as XS Imperial Stout12 Ingred...,11.00,50,90,18,144,...,102,25,164,4.123649,4.333733,4.200480,4.153661,3.879952,833,Imperial Stout (Rogue Ales)
2258,Uncle Rusty,Red Ale,Columbus Brewing Company,Columbus Brewing Company Columbus Uncle Rusty,Notes:Part of the hop odyssey.Gold medal in Ca...,9.30,55,85,17,55,...,151,8,90,4.300000,4.100000,4.100000,4.400000,4.015789,10,Uncle Rusty (Columbus Brewing Company)
955,Ruination Double IPA 2.0,IPA,Stone Brewing Co.,Stone Brewing Co. Stone Ruination IPA,Notes:Also known as Ruination in Europe.A Liqu...,8.50,65,100,21,37,...,135,0,26,4.341531,4.179179,4.183432,4.351331,4.161612,2704,Ruination Double IPA 2.0 (Stone Brewing Co.)
